# Data Analysis and Preprocessing

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder

* Load Datasets

In [5]:
fraud_data = pd.read_csv('../assets/Data/Fraud_Data.csv')
ip_to_country = pd.read_csv('../assets/Data/IpAddress_to_Country.csv')

* Handle Missing Values

In [6]:
fraud_data.fillna({'age': fraud_data['age'].median(), 'browser': 'Unknown'}, inplace=True)